<a href="https://colab.research.google.com/github/DOMINION-JOHN1/skin-lesion-detection/blob/main/RETINANET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Specify the directory path
drive_mount_point = '/content/drive/MyDrive/'

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(drive_mount_point):
    os.makedirs(drive_mount_point)

# Now, mount Google Drive to the specified directory
from google.colab import drive
drive.mount(drive_mount_point, force_remount=True)


Mounted at /content/drive/MyDrive/


In [2]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# (add --user if you don't have permission)

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ptjrz5xq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ptjrz5xq
  Resolved https://github.com/facebookresearch/detectron2.git to commit b7c7f4ba82192ff06f2bbb162b9f67b00ea55867
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6147748 sha256=8e423f7a5e931e84d7e0447130780

In [3]:
!pip install --upgrade yacs


In [4]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
import torch  # Import torch for checking CUDA availability


In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-c9b91a4a-7727-50b5-9df6-0bb0b9467e14"
#configure cuda to be very verbos
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# Check CUDA availability and the selected device
print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"Selected CUDA Device: {os.environ.get('CUDA_VISIBLE_DEVICES')}")
if torch.cuda.is_available():
    print(f"Current CUDA Device: {torch.cuda.current_device()} - {torch.cuda.get_device_name()}")

CUDA is available: True
Selected CUDA Device: MIG-c9b91a4a-7727-50b5-9df6-0bb0b9467e14
Current CUDA Device: 0 - Tesla T4


In [6]:
# ----------------------- Dataset registration -----------------------
def register_dataset(name, json_path, img_dir):
    if name not in DatasetCatalog.list():
        register_coco_instances(name, {}, json_path, img_dir)
    else:
        print(f"Dataset {name} is already registered.")

In [7]:
# Debug: List all registered datasets to ensure correct registration
def list_registered_datasets():
    print("Registered datasets:")
    for dataset_name in DatasetCatalog.list():
        print(f"- {dataset_name}")

In [8]:
import json
import os
import uuid
from glob import glob
from tqdm import tqdm
from detectron2.structures import BoxMode

def yolo_to_coco(image_dir, label_dir, output_path, dataset_type):
    images = []
    annotations = []
    # Define category mappings
    categories = [{'id': 0, 'name': 'unlabeled'}]  # Replace with your actual category names
    annotation_id = 1

    # Get all image files
    image_files = glob(os.path.join(image_dir, '*.png'))
    for image_file in tqdm(image_files, desc="Processing images"):
        # Generate a random UUID for the image ID (optional)
        image_id = str(uuid.uuid4())
        image_info = {
            'file_name': os.path.basename(image_file),
            'height': 640,
            'width': 640,
            'id': image_id  # Use the generated or original image ID
        }

        label_file = os.path.join(label_dir, os.path.basename(image_file).replace('.png', '.txt'))

        # Skip images with missing label files
        if not os.path.exists(label_file):
            print(f"Warning: Label file not found for image: {image_file}")
            continue  # Skip to the next image

        # Process the image only if the label file exists
        with open(label_file, 'r') as file:
            # Check for missing image ID within the label file
            lines = file.readlines()
            if len(lines) == 0:  # Empty label file signifies missing image ID
                print(f"Warning: Label file {label_file} is empty. Skipping image.")
                continue

            for line in lines:
                class_id, x_center, y_center, width, height = [float(x) for x in line.split()]
                # Convert from YOLO to absolute COCO format
                x_min = (x_center - width / 2) * 640
                y_min = (y_center - height / 2) * 640
                x_max = (x_center + width / 2) * 640
                y_max = (y_center + height / 2) * 640

                # Calculate area of the bounding box
                width_abs = x_max - x_min
                height_abs = y_max - y_min
                area = width_abs * height_abs

                annotation = {
                    'id': annotation_id,  # Use a unique annotation ID
                    'image_id': image_id,  # Use the same image ID as the image info
                    'category_id': int(class_id),
                    'bbox': [x_min, y_min, width_abs, height_abs],  # COCO format bounding box
                    'bbox_mode': BoxMode.XYXY_ABS,
                    'area': area,  # Include area of the bounding box
                    'iscrowd': 0
                }
                annotations.append(annotation)
                annotation_id += 1

        images.append(image_info)

    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }
    with open(output_path, 'w') as output_file:
        json.dump(coco_format, output_file)

# Assuming you have your dataset prepared and the COCO JSON file generated
# ... rest of your training code ...


In [9]:
yolo_to_coco("/content/drive/MyDrive/MyDrive/skin lesions/images/train",'/content/drive/MyDrive/MyDrive/skin lesions/labels/train',  "/content/drive/MyDrive/MyDrive/skin lesions/labels/train/annotations_train.json","train" )

Processing images: 100%|██████████| 7/7 [00:02<00:00,  2.96it/s]


In [10]:
import json
import os
import uuid
from glob import glob
from tqdm import tqdm
from detectron2.structures import BoxMode

def yolo_to_coco(image_dir, label_dir, output_path, dataset_type):
    images = []
    annotations = []
    # Define category mappings
    categories = [{'id': 0, 'name': 'unlabeled'}]  # Replace with your actual category names
    annotation_id = 1

    # Get all image files
    image_files = glob(os.path.join(image_dir, '*.png'))
    for image_file in tqdm(image_files, desc="Processing images"):
        # Generate a random UUID for the image ID (optional)
        image_id = str(uuid.uuid4())
        image_info = {
            'file_name': os.path.basename(image_file),
            'height': 640,
            'width': 640,
            'id': image_id  # Use the generated or original image ID
        }

        label_file = os.path.join(label_dir, os.path.basename(image_file).replace('.png', '.txt'))

        # Skip images with missing label files
        if not os.path.exists(label_file):
            print(f"Warning: Label file not found for image: {image_file}")
            continue  # Skip to the next image

        # Process the image only if the label file exists
        with open(label_file, 'r') as file:
            # Check for missing image ID within the label file
            lines = file.readlines()
            if len(lines) == 0:  # Empty label file signifies missing image ID
                print(f"Warning: Label file {label_file} is empty. Skipping image.")
                continue

            for line in lines:
                class_id, x_center, y_center, width, height = [float(x) for x in line.split()]
                # Convert from YOLO to absolute COCO format
                x_min = (x_center - width / 2) * 640
                y_min = (y_center - height / 2) * 640
                x_max = (x_center + width / 2) * 640
                y_max = (y_center + height / 2) * 640

                # Calculate area of the bounding box
                width_abs = x_max - x_min
                height_abs = y_max - y_min
                area = width_abs * height_abs

                annotation = {
                    'id': annotation_id,  # Use a unique annotation ID
                    'image_id': image_id,  # Use the same image ID as the image info
                    'category_id': int(class_id),
                    'bbox': [x_min, y_min, width_abs, height_abs],  # COCO format bounding box
                    'bbox_mode': BoxMode.XYXY_ABS,
                    'area': area,  # Include area of the bounding box
                    'iscrowd': 0
                }
                annotations.append(annotation)
                annotation_id += 1

        images.append(image_info)

    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }
    with open(output_path, 'w') as output_file:
        json.dump(coco_format, output_file)

# Assuming you have your dataset prepared and the COCO JSON file generated
# ... rest of your training code ...


In [11]:
yolo_to_coco("/content/drive/MyDrive/MyDrive/skin lesions/images/val",'/content/drive/MyDrive/MyDrive/skin lesions/labels/val',  "/content/drive/MyDrive/MyDrive/skin lesions/labels/val/annotations_val.json","val" )

Processing images: 100%|██████████| 5/5 [00:01<00:00,  2.92it/s]


In [12]:
register_dataset("train", "/content/drive/MyDrive/MyDrive/skin lesions/labels/train/annotations_train.json", "/content/drive/MyDrive/MyDrive/skin lesions/images/train")
register_dataset("test", "/content/drive/MyDrive/MyDrive/skin lesions/labels/val/annotations_val.json", "/content/drive/MyDrive/MyDrive/skin lesions/images/val")
list_registered_datasets()  # Call to list registered datasets for debugging

Registered datasets:
- coco_2014_train
- coco_2014_val
- coco_2014_minival
- coco_2014_valminusminival
- coco_2017_train
- coco_2017_val
- coco_2017_test
- coco_2017_test-dev
- coco_2017_val_100
- keypoints_coco_2014_train
- keypoints_coco_2014_val
- keypoints_coco_2014_minival
- keypoints_coco_2014_valminusminival
- keypoints_coco_2017_train
- keypoints_coco_2017_val
- keypoints_coco_2017_val_100
- coco_2017_train_panoptic_separated
- coco_2017_train_panoptic_stuffonly
- coco_2017_train_panoptic
- coco_2017_val_panoptic_separated
- coco_2017_val_panoptic_stuffonly
- coco_2017_val_panoptic
- coco_2017_val_100_panoptic_separated
- coco_2017_val_100_panoptic_stuffonly
- coco_2017_val_100_panoptic
- lvis_v1_train
- lvis_v1_val
- lvis_v1_test_dev
- lvis_v1_test_challenge
- lvis_v0.5_train
- lvis_v0.5_val
- lvis_v0.5_val_rand_100
- lvis_v0.5_test
- lvis_v0.5_train_cocofied
- lvis_v0.5_val_cocofied
- cityscapes_fine_instance_seg_train
- cityscapes_fine_sem_seg_train
- cityscapes_fine_instanc

In [13]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train",)
cfg.DATASETS.TEST = ("test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.INPUT.MIN_SIZE_TRAIN = (640, )
cfg.INPUT.MAX_SIZE_TRAIN = 640
cfg.INPUT.MIN_SIZE_TEST = 640
cfg.INPUT.MAX_SIZE_TEST = 640
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = [25, 50]
cfg.SOLVER.WARMUP_ITERS = 10
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.DEVICE = "cuda"
cfg.SEED = 10000
cfg.OUTPUT_DIR = './train1'
cfg.OUTPUT_ENABLED = True


In [14]:
# Debug: Print the number of classes and check device setup
print(f"Number of classes: {cfg.MODEL.ROI_HEADS.NUM_CLASSES}")
print(f"Device: {cfg.MODEL.DEVICE}")


Number of classes: 1
Device: cuda


In [15]:
# ----------------------- Training -----------------------
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[04/15 12:51:52 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

R-50.pkl: 102MB [00:00, 154MB/s]                            


[04/15 12:51:53 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[04/15 12:51:53 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up - Total num: 54


backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
backbone.top_block.p6.{bias, weight}
backbone.top_block.p7.{bias, weight}
head.bbox_pred.{bias, weight}
head.bbox_subnet.0.{bias, weight}
head.bbox_subnet.2.{bias, weight}
head.bbox_subnet.4.{bias, weight}
head.bbox_subnet.6.{bias, weight}
head.cls_score.{bias, weight}
head.cls_subnet.0.{bias, weight}
head.cls_subnet.2.{bias, weight}
head.cls_subnet.4.{bias, weight}
head.cls_subnet.6.{bias, weight}
  fc1000.{bias, weight}
  stem.conv1.bias


[04/15 12:51:53 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/15 12:52:41 d2.utils.events]:  eta: 0:28:58  iter: 19  total_loss: 2.687  loss_cls: 1.826  loss_box_reg: 0.8629    time: 1.9389  last_time: 1.9332  data_time: 0.3756  last_data_time: 0.4275   lr: 0.0025  max_mem: 10284M
[04/15 12:53:22 d2.utils.events]:  eta: 0:28:41  iter: 39  total_loss: 1.366  loss_cls: 0.9391  loss_box_reg: 0.4269    time: 1.9116  last_time: 1.7852  data_time: 0.3038  last_data_time: 0.2503   lr: 0.00025  max_mem: 10284M
[04/15 12:53:59 d2.utils.events]:  eta: 0:28:20  iter: 59  total_loss: 0.998  loss_cls: 0.6425  loss_box_reg: 0.375    time: 1.8932  last_time: 2.0955  data_time: 0.2770  last_data_time: 0.4023   lr: 2.5e-05  max_mem: 10284M
[04/15 12:54:37 d2.utils.events]:  eta: 0:28:00  iter: 79  total_loss: 0.9142  loss_cls: 0.5611  loss_box_reg: 0.3579    time: 1.8909  last_time: 1.9098  data_time: 0.2777  last_data_time: 0.2520   lr: 2.5e-05  max_mem: 10284M
[04/15 12:55:16 d2.utils.events]:  eta: 0:27:52  iter: 99  total_loss: 0.8785  loss_cls: 0.5337  l

In [16]:
# ----------------------- Evaluation -----------------------
evaluator = COCOEvaluator("test")
val_loader = build_detection_test_loader(cfg, "test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [04/15 13:24:12 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/15 13:24:12 d2.data.datasets.coco]: Loaded 5 images in COCO format from /content/drive/MyDrive/MyDrive/skin lesions/labels/val/annotations_val.json
[04/15 13:24:12 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(640, 640), max_size=640, sample_style='choice')]
[04/15 13:24:13 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/15 13:24:13 d2.data.common]: Serializing 5 elements to byte tensors and concatenating them all ...
[04/15 13:24:13 d2.data.common]: Serialized dataset takes 0.00 MiB
[04/15 13:24:13 d2.evaluation.evaluator]: Start inference on 5 batches


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[04/15 13:24:14 d2.evaluation.evaluator]: Total inference time: 0:00:00.112872 (0.112872 s / iter per device, on 1 devices)
[04/15 13:24:14 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.072730 s / iter per device, on 1 devices)
[04/15 13:24:14 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[04/15 13:24:14 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[04/15 13:24:14 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[04/15 13:24:14 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 seconds.
[04/15 13:24:14 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[04/15 13:24:14 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.00 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50      | are

OrderedDict([('bbox',
              {'AP': 0.38309968223099683,
               'AP50': 1.3179717971797178,
               'AP75': 0.0,
               'APs': 0.16771064861588197,
               'APm': 0.8201356670461174,
               'APl': nan})])